In [1]:
%load_ext autoreload
%autoreload 2

import IPython
from pathlib import Path
import os
locals = IPython.extract_module_locals() # type: ignore
notebook_name = "/".join(locals[1]["__vsc_ipynb_file__"].split("/"))
os.chdir(Path(notebook_name).parent.parent)

In [2]:
from typing import Callable, Any
from pathlib import Path

import numpy as np
import polars as pl
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import random

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset

from retail_recommender_system.utils import load_model

seed = 0
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)

In [3]:
relations = pl.read_parquet(".data/intermediate/relations.parquet")
users = pl.read_parquet(".data/intermediate/users.parquet")
items = pl.read_parquet(".data/intermediate/articles.parquet")

In [13]:
relations = relations.sample(fraction=0.2)

In [14]:
X_train, X_valid, _, _ = train_test_split(relations, np.ones(len(relations)), test_size=0.2, random_state=seed)

In [15]:
class MFDataset(Dataset):
    def __init__(self, df: pl.DataFrame, n_items: int, neg_sampl: int = 5):
        self._df = df
        self._n_items = n_items
        self._neg_sampl = neg_sampl

    def __len__(self):
        return len(self._df)
    
    def __getitem__(self, idx: int) -> dict[str, Any]:
        row = self._df[idx]
        user = torch.tensor(row.get_column("customer_id_map").to_numpy(), dtype=torch.int32)
        items = torch.tensor(row.get_column("article_id_map").to_numpy(), dtype=torch.int32)

        u_id = user.repeat(self._neg_sampl + 1)
        i_id = torch.cat([items, self._approx_neg_sampl()])
        target = torch.tensor([1.0] + [0.0] * self._neg_sampl, dtype=torch.float)

        return {
            "u_id": u_id,
            "i_id": i_id,
            "target": target
        }
    
    def _approx_neg_sampl(self):
        neg_i_id = torch.randint(low=0, high=self._n_items, size=(self._neg_sampl,), dtype=torch.int32)
        return neg_i_id
    
def _collate_fn(batch):
    u_id = torch.cat([x["u_id"] for x in batch])
    i_id = torch.cat([x["i_id"] for x in batch])
    target = torch.cat([x["target"] for x in batch])
    return {
        "u_id": u_id,
        "i_id": i_id,
        "target": target
    }

In [16]:
class MF(nn.Module):
    def __init__(self, n_users, n_items, emb_size):
        super().__init__()
        self.user_factors = nn.Embedding(n_users, emb_size)
        self.item_factors = nn.Embedding(n_items, emb_size)
        
    def forward(self, x):
        user_factors = self.user_factors(x["u_id"])
        item_factors = self.item_factors(x["i_id"])
        return (user_factors * item_factors).sum(1)

In [20]:
batch_size = 4096
neg_sampl = 3
n_users = users.get_column("customer_id_map").max()
n_items = items.get_column("article_id_map").max()

train_dataset = MFDataset(X_train, n_items=n_items, neg_sampl=neg_sampl)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=_collate_fn)

val_dataset = MFDataset(X_valid, n_items=n_items, neg_sampl=neg_sampl)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, collate_fn=_collate_fn)

embedding_size = 16
dropout_rate = 0.3
lr = 1e-4
device = torch.device("cpu")

model = MF(n_users, n_items, emb_size=embedding_size).to(device)

criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=lr)

In [21]:
def _batch_dict_to_device(batch: dict[str, Any], device: torch.device) -> dict[str, Any]:
    return {k: v.to(device) for k, v in batch.items()}

def train(
    model: nn.Module, 
    loss_fn: Callable,
    optimizer: torch.optim.Optimizer,
    train_loader: DataLoader,
    device: torch.device, 
    epoch: int, 
    print_every: None | int = None
) -> float:
    model.train()
    train_loss = 0.

    for batch_idx, batch in enumerate(train_loader):
        data = _batch_dict_to_device(batch, device)
        
        optimizer.zero_grad()
        output = model(data)
        loss = loss_fn(output, data["target"])
        loss.backward()
        optimizer.step()

        loss_item = loss.detach().cpu().item()

        if print_every is not None and batch_idx % print_every == 0:
            print(
                "Train (Batch): [{}/{} ({:.0f}%)]\tTrain Loss: {:.4f}".format(
                    batch_idx, len(train_loader), 100.0 * batch_idx / len(train_loader), loss_item
                ) # type: ignore
            )
        train_loss += loss_item

    train_loss /= len(train_loader)

    return train_loss


def test(
    model: nn.Module, 
    loss_fn: Callable,
    device: torch.device,
    test_loader: DataLoader, 
    print_every: None | int = None
) -> float:
    model.eval()
    test_loss = 0.
    with torch.no_grad():
        for batch_idx, batch in enumerate(test_loader):
            data = _batch_dict_to_device(batch, device)

            output = model(data)
            loss =  loss_fn(output, data["target"])

            loss_item = loss.detach().cpu().item()
            test_loss += loss_item

    test_loss /= len(test_loader)

    if print_every is not None:
        print(
            "\nTest: Test loss: {:.4f}".format(test_loss) # type: ignore
        )
    
    return test_loss

In [22]:
history = {
    'train_loss': [],
    'val_loss': []
}

EPOCHS = 15
for epoch in tqdm(range(1, EPOCHS + 1)):
    train(model, criterion, optimizer, train_loader, device, epoch, print_every=100)
    test(model, criterion, device, val_loader, print_every=1)
    #torch.save(model.state_dict(), f".models/mf_{epoch}.pth")

  0%|          | 0/15 [00:00<?, ?it/s]

Train (Batch): [0/1242 (0%)]	Train Loss: 1.7594
Train (Batch): [100/1242 (8%)]	Train Loss: 1.7211
Train (Batch): [200/1242 (16%)]	Train Loss: 1.7101
Train (Batch): [300/1242 (24%)]	Train Loss: 1.7815
Train (Batch): [400/1242 (32%)]	Train Loss: 1.7400
Train (Batch): [500/1242 (40%)]	Train Loss: 1.7403
Train (Batch): [600/1242 (48%)]	Train Loss: 1.7333
Train (Batch): [700/1242 (56%)]	Train Loss: 1.7589
Train (Batch): [800/1242 (64%)]	Train Loss: 1.7600
Train (Batch): [900/1242 (72%)]	Train Loss: 1.7329
Train (Batch): [1000/1242 (81%)]	Train Loss: 1.7039
Train (Batch): [1100/1242 (89%)]	Train Loss: 1.7110
Train (Batch): [1200/1242 (97%)]	Train Loss: 1.7182


  7%|▋         | 1/15 [02:37<36:42, 157.32s/it]


Test: Test loss: 1.7155
Train (Batch): [0/1242 (0%)]	Train Loss: 1.7214
Train (Batch): [100/1242 (8%)]	Train Loss: 1.7141
Train (Batch): [200/1242 (16%)]	Train Loss: 1.6930
Train (Batch): [300/1242 (24%)]	Train Loss: 1.6821
Train (Batch): [400/1242 (32%)]	Train Loss: 1.7004
Train (Batch): [500/1242 (40%)]	Train Loss: 1.6873
Train (Batch): [600/1242 (48%)]	Train Loss: 1.7477
Train (Batch): [700/1242 (56%)]	Train Loss: 1.6902
Train (Batch): [800/1242 (64%)]	Train Loss: 1.6981
Train (Batch): [900/1242 (72%)]	Train Loss: 1.6784
Train (Batch): [1000/1242 (81%)]	Train Loss: 1.6840
Train (Batch): [1100/1242 (89%)]	Train Loss: 1.6576
Train (Batch): [1200/1242 (97%)]	Train Loss: 1.6740


 13%|█▎        | 2/15 [05:12<33:47, 156.00s/it]

...


Test: Test loss: 1.5511
Train (Batch): [0/1242 (0%)]	Train Loss: 1.5408
Train (Batch): [100/1242 (8%)]	Train Loss: 1.5249
Train (Batch): [200/1242 (16%)]	Train Loss: 1.5442
Train (Batch): [300/1242 (24%)]	Train Loss: 1.5418
Train (Batch): [400/1242 (32%)]	Train Loss: 1.5637
Train (Batch): [500/1242 (40%)]	Train Loss: 1.5255
Train (Batch): [600/1242 (48%)]	Train Loss: 1.5151
Train (Batch): [700/1242 (56%)]	Train Loss: 1.5523
Train (Batch): [800/1242 (64%)]	Train Loss: 1.5190
Train (Batch): [900/1242 (72%)]	Train Loss: 1.5388
Train (Batch): [1000/1242 (81%)]	Train Loss: 1.5203
Train (Batch): [1100/1242 (89%)]	Train Loss: 1.4974
Train (Batch): [1200/1242 (97%)]	Train Loss: 1.5205


 93%|█████████▎| 14/15 [37:00<02:39, 159.92s/it]


Test: Test loss: 1.5370
Train (Batch): [0/1242 (0%)]	Train Loss: 1.5016
Train (Batch): [100/1242 (8%)]	Train Loss: 1.5270
Train (Batch): [200/1242 (16%)]	Train Loss: 1.5124
Train (Batch): [300/1242 (24%)]	Train Loss: 1.5038
Train (Batch): [400/1242 (32%)]	Train Loss: 1.5086
Train (Batch): [500/1242 (40%)]	Train Loss: 1.4946
Train (Batch): [600/1242 (48%)]	Train Loss: 1.5026
Train (Batch): [700/1242 (56%)]	Train Loss: 1.5122
Train (Batch): [800/1242 (64%)]	Train Loss: 1.4979
Train (Batch): [900/1242 (72%)]	Train Loss: 1.5018
Train (Batch): [1000/1242 (81%)]	Train Loss: 1.4962
Train (Batch): [1100/1242 (89%)]	Train Loss: 1.5099
Train (Batch): [1200/1242 (97%)]	Train Loss: 1.5190


100%|██████████| 15/15 [39:41<00:00, 158.78s/it]


Test: Test loss: 1.5247


In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(12, 5))
ax[0].plot(history['train_loss'], label='Train Loss')
ax[0].plot(history['val_loss'], label='Validation Loss')
ax[0].set_xlabel('Epochs')
ax[0].set_ylabel('Loss')
ax[0].set_title('Training and Validation Loss Over Time')

ax[1].plot(history['train_rmse'], label='Train RMSE')
ax[1].plot(history['val_rmse'], label='Validation RMSE')
ax[1].set_xlabel('Epochs')
ax[1].set_ylabel('RMSE')
ax[1].set_title('Training and Validation RMSE Over Time')

plt.legend()
plt.show()

In [23]:
from functools import partial
from torch.utils.data import DataLoader, TensorDataset
from retail_recommender_system.evaluation.prediction import recommend_k
from retail_recommender_system.evaluation.metrics import precision_k, recall_k
from retail_recommender_system.evaluation.evaluation import EvalDataset

def collate_fn(batch):
    batch = torch.cat(batch)
    return {
        "u_id": batch[:, 0],
        "i_id": batch[:, 1]
    }

@torch.no_grad
def recommend_udf(batch: dict[str, torch.Tensor], model: nn.Module, K: int = 5) -> torch.Tensor:
    model.eval()
    return model(batch).view(-1, n_items)

K = 5
n_users_eval = 1024*20
loader = DataLoader(
    EvalDataset(n_users_eval, n_items, user_batch_size=1024), 
    batch_size=1,
    collate_fn=collate_fn,
    shuffle=False
)

In [10]:
ground_truth = torch.from_numpy(
    (
        X_valid
        .filter(pl.col("customer_id_map") < n_users_eval)
        .select("customer_id_map", "article_id_map").to_numpy()
    )
).to(torch.int32).T

In [ ]:
model_config = {
    "n_users": n_users,
    "n_items": n_items,
    "emb_size": embedding_size   
}
models = [load_model(MF, Path(f".models/mf_{i}.pth"), model_config) for i in range(1, 4)]
models

In [12]:
model_base = MF(**model_config)
models.insert(0, model_base)

In [ ]:
for i, m in enumerate(models):
    recommendations = recommend_k(partial(recommend_udf, model=m, K=K), loader, K, past_interactions=None)
    precision = precision_k(recommendations, ground_truth, k=K, n_items=n_items)
    recall = recall_k(recommendations, ground_truth, k=K, n_items=n_items)
    print(f"Model epoch: {i} - Precision@{K}: {precision:.7f} - Recall@{K}: {recall:.7f}")